In [ ]:
import numpy as np
import pandas as pd
from jax import numpy as jnp
import jax

pd.options.plotting.backend = "plotly"

from summer2 import CompartmentalModel, Stratification
from summer2.parameters import Parameter, Function, Data

In [ ]:
def build_unstratified_model(settings: dict) -> CompartmentalModel:
    """
    Create a compartmental model, with the minimal compartmental structure needed to run and produce some sort of 
    meaningful outputs.
    
    Args:
        parameters: Parameters for use in model construction    
    Returns:
        A compartmental model currently without stratification applied
    """

    model = CompartmentalModel(
        [
            settings["start_time"], 
            settings["end_time"],
        ],
        [
            "susceptible", 
            "infectious",
            "recovered",
        ],
        ["infectious"],
        timestep=0.1,
    )
    model.set_initial_population(
        {
            "susceptible": Parameter("total_population") - Parameter("seed_size"), 
            "infectious": Parameter("seed_size"),
        }
    )
    model.add_infection_frequency_flow(
        "infection", 
        Parameter("contact_rate"), 
        "susceptible", 
        "infectious"
    )
    
    # Infectious people recover
    model.add_transition_flow(
        name="recovery",
        fractional_rate=Parameter("recovery_rate"),
        source="infectious",
        dest="recovered",
    )
    
    return model

In [ ]:
parameters = {
    "contact_rate": 3.0,
    "recovery_rate": 1.0,
    "total_population": 1000.,
    "seed_size": 10.,
}
settings = {
    "start_time": 0.,
    "end_time": 10.,
}
m = build_unstratified_model(settings)

m.run(parameters)
m.get_outputs_df().plot()